In [22]:
!pip install --upgrade -q langchain
!pip install google-generativeai langchain-google-genai
!pip install chromadb pypdf2 python-dotenv
!pip install PyPDF
!pip install -U langchain-community
!pip install sentence-transformers
!pip install langchainhub  

  Using cached chromadb-0.5.11-py3-none-any.whl.metadata (6.8 kB)
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.115.0-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.31.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached posthog-3.7.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.19.2-cp312-cp312-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.27.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_instrumentatio

  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


In [23]:
# Librerías Genéricas  
#from google.colab import userdata
import os
from IPython.display import Markdown

# Librerías para la preparación de datos
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

# Librerías para el proceso de Retrieval
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

In [24]:
import re
import pandas as pd
from google.cloud import bigquery
import vertexai 
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part
import vertexai.preview.generative_models as generative_models 
import sys
import os

In [25]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [26]:
# Define project information
PROJECT_ID = "ldp-data-genai-producto"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


vertexai.init(project=PROJECT_ID, location=LOCATION)

In [27]:
import requests
from langchain import LLMChain
from pydantic import BaseModel
from selenium import webdriver

In [ ]:

#from langchain_llms import Gemini

# Configuración de Gemini (asegúrate de que esta instancia es correcta)
gemini_model = GenerativeModel("gemini-1.5-pro-002")

# Definición de clases
class Libro(BaseModel):
    """Información acerca de un libro"""
    título: str
    autor: str
    precio: str

class LibroScrapper(BaseModel):
    """Scrapper de libros"""
    url: str
    html: str

def obtener_html(url: str) -> str:
    """
    Obtener HTML crudo de una página web utilizando Selenium.
    
    Args:
        url (str): URL de la página web.
    
    Returns:
        str: HTML crudo de la página web.
    """
    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source
    driver.quit()
    return html

def extraer_info(html: str) -> list[Libro]:
    """
    Extraer información de libros desde el HTML crudo utilizando Gemini.
    
    Args:
        html (str): HTML crudo de la página web.
    
    Returns:
        list[Libro]: Lista de objetos Libro con la información extraída.
    """
    
    resultado = gemini_model.generate_content(
        f"""Eres un experto en hacer web scraping y analizar HTML crudo:{html}, si no se proporciona explícitamente no supongas""",
        generation_config=GenerationConfig(
            temperature=0,
            max_output_tokens=8192,
        )
    )
    
    # Accede directamente al contenido de `resultado` si es un único objeto
    texto_generado = resultado.text  # Asegúrate de que `text` es la propiedad correcta
    
    libros = []
    
    # Suponiendo que cada libro está separado por un salto de línea
    for libro in texto_generado.split("\n"):
        datos = libro.split(",")  # Suponiendo que los datos están separados por comas
        if len(datos) == 3:
            libros.append(Libro(título=datos[0], autor=datos[1], precio=datos[2]))
    
    return libros

def main():
    url = "https://www.buscalibre.pe"
    html = obtener_html(url)
    libros = extraer_info(html)
    print(libros)

if __name__ == "__main__":
    main()



In [31]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from parsel import Selector
from webdriver_manager.chrome import ChromeDriverManager

In [37]:

gemini_model = GenerativeModel("gemini-1.5-pro-002")

# Definición de clases
class Libro(BaseModel):
    """Información acerca de un libro"""
    título: str
    autor: str
    precio: str

def obtener_html(url: str) -> str:
    """
    Obtener HTML crudo de una página web utilizando Selenium.
    
    Args:
        url (str): URL de la página web.
    
    Returns:
        str: HTML crudo de la página web.
    """
    driver_path = "C:\\chromedriver\\chromedriver-win64\\chromedriver.exe"
    service = Service(executable_path=driver_path)
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)                                                   
    driver.get(url)
    html = driver.page_source
    driver.quit()
    return html

def extraer_info(html: str) -> list[Libro]:
    """
    Extraer información de libros desde el HTML crudo utilizando Gemini.
    
    Args:
        html (str): HTML crudo de la página web.
    
    Returns:
        list[Libro]: Lista de objetos Libro con la información extraída.
    """
    
    # Prompt mejorado
    prompt = (
        "Por favor, analiza el siguiente HTML y extrae la información de los libros. "
        "Devuélveme un JSON con una lista de libros, cada uno con los campos 'título', 'autor', y 'precio':\n"
        f"{html}"
    )
    
    resultado = gemini_model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=0,
            max_output_tokens=8192,
        )
    )
    
    # Supongamos que la respuesta es un JSON string
    libros_data = resultado.text.strip()  # Limpiamos espacios en blanco
    libros = []
    
    # Parsear el JSON
    try:
        libros_json = json.loads(libros_data)
        for libro in libros_json:
            libros.append(Libro(**libro))
    except json.JSONDecodeError:
        print("Error al decodificar el JSON:", libros_data)
    
    return libros

def main():
    url = "https://www.buscalibre.pe"
    html = obtener_html(url)
    libros = extraer_info(html)
    print(libros)

if __name__ == "__main__":
    main()


ValueError: There is no such driver by url https://chromedriver.storage.googleapis.com/129.0.6668/chromedriver_win32.zip

In [14]:
import os
os.environ['CHROMEDRIVER'] = os.path.expanduser('~/chromedriver')

In [18]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from parsel import Selector
from webdriver_manager.chrome import ChromeDriverManager


In [36]:

gemini_model = GenerativeModel("gemini-1.5-pro-002")

# Definición de clases
class Libro(BaseModel):
    """Información acerca de un libro"""
    título: str
    autor: str
    precio: str

def obtener_html(url: str) -> str:
    """
    Obtener HTML crudo de una página web utilizando Selenium.
    
    Args:
        url (str): URL de la página web.
    
    Returns:
        str: HTML crudo de la página web.
    """
    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source
    driver.quit()
    return html

def extraer_info(html: str) -> list[Libro]:
    """
    Extraer información de libros desde el HTML crudo utilizando Gemini.
    
    Args:
        html (str): HTML crudo de la página web.
    
    Returns:
        list[Libro]: Lista de objetos Libro con la información extraída.
    """
    
    # Prompt mejorado
    prompt = (
        "Por favor, analiza el siguiente HTML y extrae la información de lo jugaodres de futbol. "
        "Devuélveme un JSON con una lista de distintos jugadores, cada uno con los campos 'nombre', 'equipo', y 'valor de mercado':\n"
        f"{html}"
    )
    
    resultado = gemini_model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=0,
            max_output_tokens=8192,
        )
    )
    
    # Supongamos que la respuesta es un JSON string
    libros_data = resultado.text.strip()  # Limpiamos espacios en blanco
    libros = []
    
    # Parsear el JSON
    try:
        libros_json = json.loads(libros_data)
        for libro in libros_json:
            libros.append(Libro(**libro))
    except json.JSONDecodeError:
        print("Error al decodificar el JSON:", libros_data)
    
    return libros

def main():
    url = "https://www.transfermarkt.es/navigation/marktwerte"
    html = obtener_html(url)
    libros = extraer_info(html)
    print(libros)

if __name__ == "__main__":
    main()

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [ ]:

gemini_model = GenerativeModel("gemini-1.5-pro-002")

# Definición de clases
class Libro(BaseModel):
    """Información acerca de un libro"""
    título: str
    autor: str
    precio: str

def obtener_html(url: str) -> str:
    """
    Obtener HTML crudo de una página web utilizando Selenium.
    
    Args:
        url (str): URL de la página web.
    
    Returns:
        str: HTML crudo de la página web.
    """
    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source
    driver.quit()
    return html

def extraer_info(html: str) -> list[Libro]:
    """
    Extraer información de libros desde el HTML crudo utilizando Gemini.
    
    Args:
        html (str): HTML crudo de la página web.
    
    Returns:
        list[Libro]: Lista de objetos Libro con la información extraída.
    """
    
    # Prompt mejorado
    prompt = (
        "Por favor, analiza el siguiente HTML y extrae la información de los libros. "
        "Devuélveme un JSON con una lista de libros, cada uno con los campos 'título', 'autor', y 'precio':\n"
        f"{html}"
    )
    
    resultado = gemini_model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=0,
            max_output_tokens=8192,
        )
    )
    
    # Supongamos que la respuesta es un JSON string
    libros_data = resultado.text.strip()  # Limpiamos espacios en blanco
    libros = []
    
    # Parsear el JSON
    try:
        libros_json = json.loads(libros_data)
        for libro in libros_json:
            libros.append(Libro(**libro))
    except json.JSONDecodeError:
        print("Error al decodificar el JSON:", libros_data)
    
    return libros

def main():
    url = "https://www.buscalibre.pe"
    html = obtener_html(url)
    libros = extraer_info(html)
    print(libros)

if __name__ == "__main__":
    main()


In [44]:


# Definición de clases
class Libro(BaseModel):
    """Información acerca de un libro"""
    título: str
    autor: str
    precio: str

def obtener_html(url: str) -> str:
    """
    Obtener HTML crudo de una página web utilizando Selenium.
    
    Args:
        url (str): URL de la página web.
    
    Returns:
        str: HTML crudo de la página web.
    """
    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source
    driver.quit()
    return html

def extraer_info(html: str) -> list[Libro]:
    """
    Extraer información de libros desde el HTML crudo utilizando Gemini.
    
    Args:
        html (str): HTML crudo de la página web.
    
    Returns:
        list[Libro]: Lista de objetos Libro con la información extraída.
    """
    
    # Prompt mejorado
    prompt = (
        "Por favor, analiza el siguiente HTML y extrae la información de los libros. "
        "Devuélveme un JSON con una lista de libros, cada uno con los campos 'título', 'autor', y 'precio':\n"
        f"{html}"
    )
    
    resultado = gemini_model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=0,
            max_output_tokens=8192,
        )
    )
    
    # Supongamos que la respuesta es un JSON string
    libros_data = resultado.text.strip()  # Limpiamos espacios en blanco
    libros = []
    
    # Intentar cargar el JSON
    try:
        libros_json = json.loads(libros_data)
        libros = [Libro(**libro) for libro in libros_json]  # Crear lista de libros
    except json.JSONDecodeError as e:
        print("Error al decodificar el JSON:", e)
        print("Respuesta del modelo:", libros_data)
    
    return libros

def main():
    url = "https://www.buscalibre.pe"
    html = obtener_html(url)
    libros = extraer_info(html)
    print(libros)

if __name__ == "__main__":
    main()


Error al decodificar el JSON: Expecting value: line 1 column 1 (char 0)
Respuesta del modelo: ```json
[]
```

El HTML proporcionado no contiene información de libros en un formato que pueda ser fácilmente extraído para crear el JSON deseado.  El HTML muestra la página principal de Buscalibre Perú, con banners, menús de navegación y algunas sugerencias de libros, pero los datos de 'título', 'autor' y 'precio' están embebidos dentro de la estructura HTML de una manera que requiere un análisis más complejo (parsing) con una herramienta específica o una librería de web scraping.

Para extraer la información de los libros, necesitarías usar una librería como Beautiful Soup (en Python) o Cheerio (en Node.js) para analizar el HTML y extraer los datos de los elementos relevantes (etiquetas `<h3>` para el título, `<div class="autor">` para el autor y `<p class="precio-ahora">` para el precio).

**Ejemplo con Beautiful Soup (Python):**

```python
import requests
from bs4 import BeautifulSoup
imp